In [1]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()
sc =SparkContext()
sqlCtx = SQLContext(sc)


In [2]:
# Campaign details from MARK 
rdd = sc.textFile("gs://ds-url-catag/campaign_analysis/campaign_details.csv") \
    .map(lambda line: line.split(";"))     
first_line = rdd.first()
campaigns_list = rdd.filter(lambda line: line != first_line).toDF(['orders_id','order','audseg_idname','audseg_id','impr','click','category'])


In [3]:
#details of the campaigns - This has to be removed 
campaign_details = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/processed_orders/")

#tagging the campagin list - the campaings that use our model
campaigns_list = campaigns_list.withColumn("To_check",lit(True))

#get the details of the campagins list
cond = [campaign_details.order_id == campaigns_list.orders_id,campaign_details.target_audienceid == campaigns_list.audseg_id]

#getting the detailds of all the campaign list
campaignslisttier1 = campaigns_list.join(campaign_details,cond,'left')

In [4]:
#reading all the campaign details obtained from datapreprocessing and scraping - previous two steps 
filtered_campaigns = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/campaign_details/")
filtered_campaigns = filtered_campaigns.select(col("order_id").alias("orderid"),col("Order_name"),col("IABTier1Categorization"),col("IOEndDate"),col("IOStartDate"))

#get the details of the filtered campaigns
cond = [campaign_details.order_id == filtered_campaigns.orderid]

#getting the details of all the campaign list
filtcamptier1 = filtered_campaigns.join(campaign_details,cond,'left')

#To filter the campaigns before applying model "TO_check !=  True"
filtcamptier2 = filtcamptier1.join(campaigns_list,filtcamptier1.orderid == campaigns_list.orders_id,'left')
filtcamptier2 =filtcamptier2.withColumn("To_check",when(col('To_check') == True,col('To_check')).otherwise(False))
filtcamptier3 = filtcamptier2.where(col('To_check') == False )

In [11]:
#comparing campaigns 
from pyspark.sql.types import BooleanType,StringType
import re
#'DBS',indosat','grab','Google roadblock','uber','opera']

#GOJEK
def regex_filter(x):
    list_campaign = ['grab']
    filter_status = ''
    string_x = x[:5]
    for camp in list_campaign:
        if camp.lower() in string_x.lower():
            filter_status = True
    return filter_status
filter_udf = udf(regex_filter, BooleanType())

#Before application of the model
filtcamptier3 = filtcamptier3.withColumn('Filter_campaigns',filter_udf(filtcamptier3.Order_name))    

In [46]:
#gojek old
go_jek_old = filtcamptier3.where(col('Filter_campaigns') == True)
go_jek_old = go_jek_old.withColumn("filter",when((col('Age_group_1') != "null"),False).otherwise(True))
go_jek_old = go_jek_old.where(col('filter') == True)
go_jek_old = go_jek_old.withColumn("CTR",((col('clicks')/col('impressions'))*100))
old = go_jek_old.where((col('CTR') != 0) & (col('CTR').isNotNull()))
# # go_jek_final = go_jek_final.withColumn("CTR",((col('clicks')/col('impressions'))*100))

In [15]:
#gojek new
campaignslisttier1 = campaignslisttier1.withColumn('Filter_campaigns',filter_udf(campaignslisttier1.order))
go_jek_new = campaignslisttier1.where(col('Filter_campaigns') == True)
# go_jek_new = go_jek_new.withColumn("filter",when(col('Age_group_1') != "null",True).otherwise(False))
# new = go_jek_new.where(col('filter') == True)

In [48]:
#calculating statistics
#no of campaigns
print old.count()

#calculate the difference between timestamps
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd"
timeDiff = (F.unix_timestamp('IOEndDate', format=timeFmt)
            - F.unix_timestamp('IOStartDate', format=timeFmt))
#for old
old = old.withColumn("Duration", (((timeDiff/60)/60/24)))

# #avg no of days 
print old.agg(avg(col('Duration'))).show()

#impressions and clicks 
old = old.withColumn("CTR",((col('clicks')/col('impressions'))*100))

#CTR calculation - check the conditions to filter 
rem_out = old.where(col('CTR').isNotNull())
print rem_out.agg(avg(col('CTR'))).show()


#Average Budget 
print old.agg(avg(col('budget'))).show(1,False)

print old.groupby('target_country').count().show()

73
+------------------+
|     avg(Duration)|
+------------------+
|143.02739726027397|
+------------------+

None
+------------------+
|          avg(CTR)|
+------------------+
|2.2272218752654616|
+------------------+

None
+---------------------+
|avg(budget)          |
+---------------------+
|1.3511590532154438E13|
+---------------------+

None
+---------------+-----+
| target_country|count|
+---------------+-----+
|        Jakarta|    1|
|     Banyuwangi|    2|
|          Bogor|    8|
| Bekasi Regency|    1|
|          Depok|   10|
|           Bali|    1|
|         Bekasi|   19|
|      Indonesia|    2|
|        Bandung|    6|
|South Tangerang|   23|
+---------------+-----+

None


In [41]:
# #no of campaigns
# print go_jek_new.count()

# #calculate the difference between timestamps
# from pyspark.sql import functions as F
# timeFmt = "yyyy-MM-dd"
# timeDiff = (F.unix_timestamp('End_date', format=timeFmt)
#             - F.unix_timestamp('start_date', format=timeFmt))
# #for old
# go_jek_new = go_jek_new.withColumn("Duration", (((timeDiff/60)/60/24)))

# #avg no of days 
# print go_jek_new.agg(avg(col('Duration'))).show()

# # impressions and clicks 
# go_jek_new = go_jek_new.withColumn("CTR",((col('clicks')/col('impressions'))*100))

# #CTR calculation - check the conditions to filter 
# go_jek_new = go_jek_new.where(col('CTR') !=0)
# print go_jek_new.agg(avg(col('CTR'))).show()

# #Average Budget 
# print go_jek_new.agg(avg(col('budget'))).show(1,False)


# print go_jek_new.groupby('target_country').count().show()

In [43]:
old.select(['orderid','Order_name','ad_type_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','Age_group_5','CTR']).show(1300)

+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+-------------------+
|   orderid|          Order_name|ad_type_1|Age_group_1|Age_group_2|Age_group_3|Age_group_4|Age_group_5|                CTR|
+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+-------------------+
| 451618836|Grab_20161227_201...|     diau|       null|       null|       null|       null|       null|                0.0|
| 451618836|Grab_20161227_201...|     diau|       null|       null|       null|       null|       null|                0.0|
| 451618836|Grab_20161227_201...|     diau|       null|       null|       null|       null|       null|                0.0|
|2119603376|Grab_20170812_201...|     diau|       null|       null|       null|       null|       null|                0.0|
|2119603376|Grab_20170812_201...|     diau|       null|       null|       null|       null|       null|               null|
|2119603

In [39]:
# go_jek_new.select(['orders_id','order','ad_type_1','Age_group_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','Age_group_5']).show()

In [51]:
new.select(['orders_id','order','ad_type_1','Age_group_1','CTR']).show(55)

+----------+--------------------+---------+-----------+------------------+
| orders_id|               order|ad_type_1|Age_group_1|               CTR|
+----------+--------------------+---------+-----------+------------------+
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44| 0.623251384010953|
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44|0.7881480642970342|
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44| 0.839585751754242|
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44|0.7689039932030586|
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44|0.9423430522721302|
|2180363900|DBS-EXC_20171107_...|     dnau|      35-44|0.6171376291768033|
|2191042068|DBS-Bandung-Audie...|     dnau|      25-34|0.9405624042708225|
|2191042068|DBS-Bandung-Audie...|     dnau|      25-34|1.0806782187441772|
|2195762072|DBS_20171129_2017...|     dnau|      35-44|0.8551919673190004|
|2195762072|DBS_20171129_2017...|     dnau|      35-44|1.1148272017837235|
|2199386871|DBS_20171205_